In [0]:
df = spark.read.format('delta').load('abfss://bronze@databricksetestg.dfs.core.windows.net/customers')
display(df)

In [0]:
from pyspark.sql.functions import *
df = df.withColumn('domain',split(col('email'),'@').getItem(1))
df = df.drop('_rescued_data')
display(df)

In [0]:
df = df.groupBy("domain").agg(count("customer_id").alias("total_customers")).sort(desc("total_customers"))
display(df)

In [0]:
df = df.withColumn("full_name",concat(col("first_name"),lit(" "),col("last_name")))
df = df.drop("first_name","last_name")
display(df)

In [0]:
df.write.format("delta").mode("overwrite").save("abfss://silver@databricksetestg.dfs.core.windows.net/customers")

In [0]:
%sql
CREATE TABLE IF NOT EXISTS prem_catalog.silver.customers_silver
USING DELTA
LOCATION 'abfss://silver@databricksetestg.dfs.core.windows.net/customers'